In [3]:

!pip install openeo shapely rasterio opencv-python-headless kaggle --quiet

import os
import cv2
import csv
import json
import zipfile
import rasterio
import numpy as np
from shapely.geometry import box
import openeo
from google.colab import files

# Kaggle API
uploaded = files.upload()
kaggle_json_filename = list(uploaded.keys())[0]
os.makedirs("/root/.kaggle", exist_ok=True)
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(uploaded[kaggle_json_filename].decode())
os.chmod("/root/.kaggle/kaggle.json", 600)


!kaggle datasets download -d shambac/tum-sentinel-1-2 --unzip -p sentinel_data

# Openeo Authentication
connection = openeo.connect("openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

# Matching image function
def match_images(target_image, sentinel_image):
    img1 = cv2.imread(target_image, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(sentinel_image, cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    if des1 is None or des2 is None:
        return 0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    return len(matches)

def extract_coordinates(geotiff_path):
    with rasterio.open(geotiff_path) as dataset:
        bounds = dataset.bounds
        return bounds.top, bounds.left, bounds.bottom, bounds.right

def get_hemisphere(lat):
    return "Northern Hemisphere" if lat >= 0 else "Southern Hemisphere"

def get_temperature_zone(lat):
    if abs(lat) < 23.5:
        return "Tropical Zone"
    elif abs(lat) < 66.5:
        return "Temperate Zone"
    else:
        return "Polar Zone"

def download_openeo_geotiff(bbox, time_range, output_file):
    cube = connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent={
            "west": bbox.bounds[0],
            "south": bbox.bounds[1],
            "east": bbox.bounds[2],
            "north": bbox.bounds[3],
            "crs": "EPSG:4326"
        },
        temporal_extent=time_range,
        bands=["B04", "B03", "B02"]
    ).reduce_dimension(dimension="t", reducer="median")
    cube.download(output_file, format="GTiff")

# Setting up seasons
seasons = {
    "fall":   ["2017-09-01", "2017-11-30"],
    "spring": ["2017-03-01", "2017-05-30"],
    "summer": ["2017-06-01", "2017-08-31"],
    "winter": ["2016-12-01", "2017-02-28"]
}

data_root = "sentinel_data"
output_root = "output_csvs"
os.makedirs(output_root, exist_ok=True)

# Looping through images for matching
for season in ["fall", "spring", "summer", "winter"]:
    print(f"\nProcessing season: {season}")
    season_path = os.path.join(data_root, f"Sentinel_{season}")
    if not os.path.exists(season_path):
        print(f"Folder not found: {season_path}")
        continue

    for roi_folder in os.listdir(season_path):
        roi_path = os.path.join(season_path, roi_folder)
        if not os.path.isdir(roi_path): continue

        for serial_folder in os.listdir(roi_path):
            serial_path = os.path.join(roi_path, serial_folder)
            if not os.path.isdir(serial_path): continue

            for image_file in os.listdir(serial_path):
                if not image_file.endswith(".png"): continue
                image_path = os.path.join(serial_path, image_file)
                print(f"Matching: {image_file}")

                # matching globally using a sliding bbox
                best_score = 0
                best_bbox = None
                best_tile_lat = None
                best_tile_lon = None
                temp_image_path = "openeo_temp.png"

                for lat in range(-60, 61, 5):
                    for lon in range(-180, 181, 5):
                        try:
                            bbox = box(lon - 0.1, lat - 0.1, lon + 0.1, lat + 0.1)
                            cube = connection.load_collection(
                                "SENTINEL2_L2A",
                                spatial_extent={
                                    "west": bbox.bounds[0],
                                    "south": bbox.bounds[1],
                                    "east": bbox.bounds[2],
                                    "north": bbox.bounds[3],
                                    "crs": "EPSG:4326"
                                },
                                temporal_extent=seasons[season],
                                bands=["B04", "B03", "B02"]
                            ).reduce_dimension(dimension="t", reducer="median")

                            cube.download(temp_image_path, format="PNG")
                            score = match_images(image_path, temp_image_path)

                            if score > best_score:
                                best_score = score
                                best_bbox = bbox
                                best_tile_lat = lat
                                best_tile_lon = lon
                        except Exception as e:
                            continue

                if best_bbox and best_score > 0:
                    geotiff = f"matched_{season}_{image_file}.tif"
                    download_openeo_geotiff(best_bbox, seasons[season], geotiff)
                    top, left, bottom, right = extract_coordinates(geotiff)
                    hemisphere = get_hemisphere(top)
                    temp_zone = get_temperature_zone(top)

                    # Write CSV
                    csv_folder = os.path.join(output_root, season)
                    os.makedirs(csv_folder, exist_ok=True)
                    csv_file = os.path.join(csv_folder, "match_output.csv")

                    with open(csv_file, mode="a", newline="") as f:
                        writer = csv.writer(f)
                        if os.stat(csv_file).st_size == 0:
                            writer.writerow(["target_image", "matched_lat", "matched_lon", "hemisphere", "temperature_zone"])
                        writer.writerow([image_file, top, left, hemisphere, temp_zone])

                    os.remove(geotiff)
                    print(f"Matched: {image_file} → Score: {best_score}")
                else:
                    print(f"No match found for: {image_file}")


Saving kaggle.json to kaggle (1).json
Dataset URL: https://www.kaggle.com/datasets/shambac/tum-sentinel-1-2
License(s): CC-BY-SA-4.0
^C
Authenticated using refresh token.

🔄 Processing season: fall
⚠️ Folder not found: sentinel_data/Sentinel_fall

🔄 Processing season: spring
⚠️ Folder not found: sentinel_data/Sentinel_spring

🔄 Processing season: summer
⚠️ Folder not found: sentinel_data/Sentinel_summer

🔄 Processing season: winter
⚠️ Folder not found: sentinel_data/Sentinel_winter
